In [1]:
0==0.0

True

eigenvalues can be calculated using lin alg package or panda?

2a can be done using modules

2c use histograms to find the wavelike graph, look at the peaks of values, use them to something

2e should look like the image example in the handout

